In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


---------------------------------------------------------------

root: C:\sesa\covid19datascience\bulletin
parent: C:\sesa\covid19datascience
default_input: C:\sesa\covid19datascience\input
default_output: C:\sesa\covid19datascience\output

---------------------------------------------------------------



In [44]:
gal = pd.read_csv(join(default_input,'gal.csv'),
                  usecols=['paciente','data_nascimento','nome_mae','Requisicao','codigo_amostra','data_liberacao','IBGE.x'],
                  converters={
                      'Requisicao': lambda x: str(int(x)),
                      'codigo_amostra': lambda x: str(int(x))
                  },
                  parse_dates=['data_nascimento']
                 )
# gal = gal.drop_duplicates('Requisicao')

gal['data_liberacao'] = gal['data_liberacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y'))
gal['hash_mae'] = gal['paciente'].apply(normalize_hash) + gal['nome_mae'].apply(normalize_hash)
gal['hash_nasc'] = gal['paciente'].apply(normalize_hash) + gal['data_nascimento'].apply(date_hash)

gal

,Requisicao,data_liberacao,codigo_amostra,IBGE.x,paciente,data_nascimento,nome_mae,hash_mae,hash_nasc
0,210206138856,2021-04-09,412124312,4106902,MAXILIANO SANTOS OLIVEIRA,1972-04-05,MARIA DIONARA DOS SANTOS,MAXILIANOSANTOSOLIVEIRAMARIADIONARADOSSANTOS,MAXILIANOSANTOSOLIVEIRA05041972
1,210206136412,2021-04-08,412102614,4106902,EDINA PAIXAO DA SILVA DOS SANTOS,1968-04-12,TEREZA DE ARAUJO MONTEIRO,EDINAPAIXAODASILVADOSSANTOSTEREZADEARAUJOMONTEIRO,EDINAPAIXAODASILVADOSSANTOS12041968
2,210206134044,2021-04-05,412084411,4106902,AMARILDO SOUZA DOS REIS,1975-06-10,LENIR SANTOS DE SOUZA,AMARILDOSOUZADOSREISLENIRSANTOSDESOUZA,AMARILDOSOUZADOSREIS10061975
3,210206136988,2021-04-08,412107344,4106902,LUIZ FERNANDO DA COSTA,1989-02-24,MARIA APARECIDA DOS SANTOS COSTA,LUIZFERNANDODACOSTAMARIAAPARECIDADOSSANTOSCOSTA,LUIZFERNANDODACOSTA24021989
4,210206140345,2021-04-11,412137001,4106902,JULIANA VIEIRA DIAS,1990-06-28,KATIA REGINA VIEIRA DIAS,JULIANAVIEIRADIASKATIAREGINAVIEIRADIAS,JULIANAVIEIRADIAS28061990
...,...,...,...,...,...,...,...,...,...
287,211503019674,2021-04-06,412074995,4115200,ANALICIA DE OLIVEIRA CRISOSTOMO,1951-01-24,MARIA BARBOSA DE OLIVEIRA,ANALICIADEOLIVEIRACRISOSTOMOMARIABARBOSADEOLIV...,ANALICIADEOLIVEIRACRISOSTOMO24011951
288,211503019674,2021-04-06,412074995,4115200,ANALICIA DE OLIVEIRA CRISOSTOMO,1951-01-24,MARIA BARBOSA DE OLIVEIRA,ANALICIADEOLIVEIRACRISOSTOMOMARIABARBOSADEOLIV...,ANALICIADEOLIVEIRACRISOSTOMO24011951
289,211503019674,2021-04-06,412074995,4115200,ANALICIA DE OLIVEIRA CRISOSTOMO,1951-01-24,MARIA BARBOSA DE OLIVEIRA,ANALICIADEOLIVEIRACRISOSTOMOMARIABARBOSADEOLIV...,ANALICIADEOLIVEIRACRISOSTOMO24011951
290,211503019674,2021-04-06,412074995,4115200,ANALICIA DE OLIVEIRA CRISOSTOMO,1951-01-24,MARIA BARBOSA DE OLIVEIRA,ANALICIADEOLIVEIRACRISOSTOMOMARIABARBOSADEOLIV...,ANALICIADEOLIVEIRACRISOSTOMO24011951


In [47]:
gal.groupby(['Requisicao','hash_mae','hash_nasc'])[['paciente']].count().sort_values('paciente',ascending=False)

,,,paciente
Requisicao,hash_mae,hash_nasc,
211703055232,ISABELRODRIGUESDEPAULAROSAFERNANDESDASILVA,ISABELRODRIGUESDEPAULA02041968,7
211001014860,MARIELLEPASSOSSILVABEATRIZTEREZINHAPASSOSDASILVA,MARIELLEPASSOSSILVA23041987,6
210206139722,SEBASTIAOCARLOSDEANDRADEMARIADELOURDESPORFIRO,SEBASTIAOCARLOSDEANDRADE19011963,6
211503019819,SOLANGEAPARECIDAOLIVEIRADOSSANTOSFAVAILIRIARODRIGUESDOSSANTOS,SOLANGEAPARECIDAOLIVEIRADOSSANTOSFAVA08061970,5
211503020495,ROTSENISIDIODESENACAJADOALDENORASILVINOCAJADO,ROTSENISIDIODESENACAJADO28011945,5
...,...,...,...
211503020735,MAURADONASCIMENTOMATEUSNI,MAURADONASCIMENTOMATEUS13021950,1
210206136988,LUIZFERNANDODACOSTAMARIAAPARECIDADOSSANTOSCOSTA,LUIZFERNANDODACOSTA24021989,1
211601023867,SIDEMARDASILVAALBENAZTEREZINHAGONCALVESDASILVAALBENAZ,SIDEMARDASILVAALBENAZ07051976,1
